If the data is not split

In [23]:
from data_splitter import DataSplitter

# # Split rviz selected data
# ground_truth_name = "pass2.yaml"
test_alg_name = "pass2_alg_fast.yaml"

# splitter = DataSplitter("./data", ground_truth_name)
algorithm_splitter = DataSplitter("./data", test_alg_name)

# splitter.split_rviz_selection_data()
algorithm_splitter.split_node_data()


./data/pass2_alg_fast_split/1626350539_381778000
./data/pass2_alg_fast_split/1626350539_482635000
./data/pass2_alg_fast_split/1626350539_583515000
./data/pass2_alg_fast_split/1626350539_684358000
./data/pass2_alg_fast_split/1626350539_785221000
./data/pass2_alg_fast_split/1626350539_886077000
./data/pass2_alg_fast_split/1626350539_986943000
./data/pass2_alg_fast_split/1626350540_87796000
./data/pass2_alg_fast_split/1626350540_188658000
./data/pass2_alg_fast_split/1626350540_289518000
./data/pass2_alg_fast_split/1626350540_390366000
./data/pass2_alg_fast_split/1626350540_491227000
./data/pass2_alg_fast_split/1626350540_592089000
No objects found at timestamp:
{'timestamp': {'sec': 1626350540, 'nanosec': 592089000}, 'durations': {'normalization': 0.023803, 'conveyor_clusterization': 0, 'conveyor_classification': 0, 'density_segmentation': 0.001717, 'supports_clusterization': 8.9e-05, 'supports_classification': 0, 'estimation': 0, 'processing': 0.025609}}
./data/pass2_alg_fast_split/16263

Load data

In [1]:
import numpy as np
from data_loader import DataLoader
from feature import get_features_from_loader

tp_loader = DataLoader("./data", "pass1")
tp_features = get_features_from_loader(tp_loader)

alg_loader = DataLoader("./data", "pass1_alg_fast")
alg_features = get_features_from_loader(alg_loader)

Plat Data at timestamp Group data


In [3]:
from collections import defaultdict
from matplotlib.patches import Rectangle, Circle
import matplotlib.pyplot as plt
from feature import group_features_by_timestamp
from plot_utils import plot_feature_with_bbox
from feature import group_features_by_timestamp, distance
import os
 
def is_point_in_bounding_box(point, box_size, bbox_center) -> bool:
    print (f"Point: {point[0]}, {point[1]}, bbox_center: {bbox_center[0]}, {bbox_center[1]}, {distance(bbox_center, point)}")
    return abs(distance(bbox_center, point)) < 0.2

def plot_feature(ax, features, label, color):
    mean_points_x = []
    mean_points_y = []
    mean_points_z = []
    for feature in features:
        mean_points_x.append(feature.box_center[0])
        mean_points_y.append(feature.box_center[1])
        mean_points_z.append(feature.box_center[2])
    ax.plot(mean_points_x, mean_points_y, ".", c=color, label=label, markersize=5)


def plot_feature_with_bbox(ax, features, label, color, bbox_color):
    mean_points_x = []
    mean_points_y = []
    mean_points_z = []
    for feature in features:
        mean_points_x.append(feature.mean_point[0])
        mean_points_y.append(feature.mean_point[1])
        mean_points_z.append(feature.mean_point[2])
        center = (
            feature.box_center[0] - feature.box_size[0] / 2.0,
            feature.box_center[1] - feature.box_size[1] / 2,
        )
        ax.add_patch(
            Circle(
                center,
                0.2,
                edgecolor=bbox_color,
                facecolor=None,
                alpha=0.08,
            )
        )
    ax.plot(mean_points_x, mean_points_y, ".", c=color, label=label, markersize=3)

timestamp = "1626350547_652271000"

def plot_compare_at_timestamp(timestamp):

    # return timestamp_dict
    grouped_features = group_features_by_timestamp(tp_features, alg_features)
    matched_alg = []
    alg_false = []
    tp_true = []

    grouped_features = dict(sorted(grouped_features.items()))
    features_at_timestamp = grouped_features[timestamp]

    selected_features_at_this_time = features_at_timestamp["tp"]
    algorithm_features_at_this_time = features_at_timestamp["alg"]

    for tp_feature in selected_features_at_this_time:
        for alg_feature in algorithm_features_at_this_time:
            if alg_feature.label == "matched":
                continue

            if is_point_in_bounding_box(
                alg_feature.mean_point, tp_feature.box_size, tp_feature.box_center
            ):
                tp_feature.label = "matched"
                alg_feature.label = "matched"
                break

    for alg_feature in algorithm_features_at_this_time:
        if alg_feature.label is None:
            alg_feature.label = "false"

    for tp_feature in selected_features_at_this_time:
        if tp_feature.label is None:
            tp_feature.label = "missed"




    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(f"At {timestamp}")
    ax.grid()
    # Set plot limits
    ax.set_xlim(0, 5)
    ax.set_ylim(-2.5, 2.5)

    # Set labels
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    matched_alg = [feature for feature in algorithm_features_at_this_time if feature.label == "matched"]
    matched_rviz = [feature for feature in selected_features_at_this_time if feature.label == "matched"]
    false_alg = [feature for feature in algorithm_features_at_this_time if feature.label == "false"]
    missed = [feature for feature in selected_features_at_this_time if feature.label == "missed"]

    # plot_feature_with_bbox(ax, selected_features_at_this_time, f"Selected by hand ({len(selected_features_at_this_time)})", "g", "g")
    plot_feature(ax, matched_alg, f"Algorithm Matched ({len(matched_alg)})", "lightblue")
    plot_feature_with_bbox(ax, matched_rviz, f"Selected Matched ({len(matched_rviz)})", "darkblue", "darkblue")
    plot_feature(ax, false_alg, f"Algorithm false positives({len(false_alg)})", "red")
    plot_feature_with_bbox(ax, missed, f"Missed ({len(missed)})", "green", "green")
    ax.legend(loc="lower center")
    fig.savefig(f"/tmp/plots/{timestamp}.png", dpi=150)


wrong_list = ['1626350368_224200000', '1626350368_22476000', '1626350368_930224000', '1626350371_855108000', '1626350372_56855000', '1626350373_166271000', '1626350373_670721000', '1626350373_973179000', '1626350375_284318000', '1626350377_402348000', '1626350378_612653000', '1626350399_591308000', '1626350400_801604000', '1626350406_147138000', '1626350407_660013000', '1626350408_466885000', '1626350408_668603000', '1626350408_870337000', '1626350409_576334000', '1626350409_979768000', '1626350410_181498000', '1626350410_383371000', '1626350410_584925000', '1626350410_685783000', '1626350410_80629000', '1626350410_887850000', '1626350411_190082000', '1626350411_492656000', '1626350411_593501000', '1626350411_795226000', '1626350411_89216000', '1626350411_896086000', '1626350411_996953000', '1626350411_996953000', '1626350412_299515000', '1626350412_97826000', '1626350417_745905000', '1626350418_754494000', '1626350423_495032000', '1626350424_301734000', '1626350424_301734000', '1626350426_120182000', '1626350426_924068000', '1626350429_949833000', '1626350432_773883000', '1626350432_874743000', '1626350438_220288000', '1626350441_952037000', '1626350447_499292000', '1626350448_608734000', '1626350448_911345000', '1626350450_928492000', '1626350452_844817000', '1626350453_349133000', '1626350479_168992000', '1626350479_370735000', '1626350479_471637000', '1626350479_572531000', '1626350481_287025000', '1626350481_387880000', '1626350481_488745000', '1626350481_589601000', '1626350481_690467000', '1626350481_791327000', '1626350481_892198000', '1626350494_398695000', '1626350497_727047000', '1626350498_332190000', '1626350498_634780000', '1626350498_836621000', '1626350499_139447000', '1626350499_643361000', '1626350501_559680000', '1626350501_660538000', '1626350501_862259000', '1626350501_963413000', '1626350502_366545000', '1626350502_467426000', '1626350502_568271000', '1626350502_770002000', '1626350502_870854000', '1626350502_870854000', '1626350502_971704000', '1626350503_475996000', '1626350503_475996000', '1626350503_677711000', '1626350503_677711000', '1626350503_677711000', '1626350503_882738000', '1626350504_182020000', '1626350504_384011000', '1626350504_787161000', '1626350504_81164000', '1626350504_988879000', '1626350509_124100000', '1626350509_426747000', '1626350509_527560000', '1626350509_729254000', '1626350510_132758000', '1626350510_31827000', '1626350510_435270000']

# for wrong in wrong_list:
#     plot_compare_at_timestamp(wrong_list)


selected_timestamps_path = f"./data/pass1_split"
alg_timestamps_path = f"./data/pass1_alg_fast_split"
selected_timestamps = [folder for folder in os.listdir(selected_timestamps_path) if os.path.isdir(os.path.join(selected_timestamps_path, folder))]
alg_timestamps = [folder for folder in os.listdir(alg_timestamps_path) if os.path.isdir(os.path.join(alg_timestamps_path, folder))]
# print(selected_timestamps)
# print(alg_timestamps)

i = 0

for timestamp in selected_timestamps:
    if timestamp not in alg_timestamps:
        i+=1
        print(timestamp)
        
print(i)
i = 0

for timestamp in alg_timestamps :
    if timestamp not in selected_timestamps:
        print(timestamp)
        i+=1

print(i)

# Selected matched - lightblue
# Selected missed - green
# Algorithm matched - darkblue
# Algorithm false positives - red




Found dict_keys([]) timestamps


TypeError: unhashable type: 'list'